In [ ]:
from IPython.core.display import HTML
with open('style.css') as file:
    css = file.read()
HTML(css)

In [ ]:
# Autload python modules by default
%load_ext autoreload
%autoreload 2

# Convert notebooks to python, so they can be loaded effiently
from utils.jupyter_loader import JupyterLoader

loader = JupyterLoader()
loader.load_all()

# Prototype 1 

In the last chapters many ideas have been introduced and the engine was successively improved. One consequence though is that the necessary code is split across multiple files. The goal of this chapter is to show one complete implementation of the current engine. Additionally, some improvement is done to the `play` function. Up to now it was just using `analyse` internally and would calculate an exact score for every possible move. But as we are only interested in the best move, there is some room for improvement here. Furthermore, we will also add functionality to correctly determine the number of moves for a mate

We define a new engine `PrototypeV1Engine` that inherits the base interface `Engine`. The constructor takes an evaluator `evaluator` as parameter and the maximal look ahead depth `max_look_ahead_depth`. The member `current_depth` will keep track of the current depth we are in the search tree and is used to implement the mate functionality.

In [ ]:
from converted_notebooks.s04_engine_interface import Engine
from converted_notebooks.s11_iterative_deepening import AlphaBetaCache
from converted_notebooks.s12_simplified_evaluation import IncrementalEvaluator


class PrototypeV1Engine(Engine):

    def __init__(
        self, evaluator: IncrementalEvaluator, max_look_ahead_depth: int
    ):
        self.evaluator = evaluator
        self.max_look_ahead_depth = max_look_ahead_depth
        self.current_depth = -1
        self.cache = AlphaBetaCache()

A small auxiliary function `_adjust_mate` is defined, which takes a score `score` as parameter and returns it with the correct number of moves for a mate.

In [ ]:
from chess.engine import PovScore


def _adjust_mate(self, score: PovScore) -> PovScore:
    if score.is_mate():
        score.relative = score.relative.increased_mate(self.current_depth)
    return score


PrototypeV1Engine._adjust_mate = _adjust_mate

Then a decorator `depth_tracker` is defined, which increases `current_depth` by one before entering the function and decreases it again after the function call. This way `current_depth` will always contain the correct depth.

In [ ]:
def depth_tracker(func):

    def inner(self, *args, **kwargs):
        self.current_depth += 1
        result = func(self, *args, **kwargs)
        self.current_depth -= 1
        return result

    return inner


PrototypeV1Engine.depth_tracker = depth_tracker

The `_value` function is the same as before except that is decorated with the `depth_tracker` decorator and correctly calculates mates returned by the evaluator with the `_adjust_mate` function.

In [ ]:
import chess
from chess.engine import Score
from converted_notebooks.s11_iterative_deepening import cache_alpha_beta
from converted_notebooks.s12_simplified_evaluation import DetailedMove


@PrototypeV1Engine.depth_tracker
@cache_alpha_beta
def _value(
    self, board: chess.Board, depth: int, alpha: Score, beta: Score
) -> PovScore:
    if (score := self.evaluator.evaluate(board)) is not None:
        return self._adjust_mate(score)
    if depth == 0:
        return self._quiescence(board, alpha, beta)

    for move in board.legal_moves:
        detailed_move = DetailedMove(board, move)

        self.evaluator.push(detailed_move)
        board.push(move)
        value = self._value(board, depth - 1, -beta, -alpha).pov(not board.turn)
        board.pop()
        self.evaluator.pop()

        if value >= beta:
            return PovScore(value, board.turn)
        alpha = max(alpha, value)

    return PovScore(alpha, board.turn)


PrototypeV1Engine._value = _value

Similarly, the `_quiescence` method is the same as before, but decorated with `depth_tracker`. The optimization functions for the `_quiescence` search have not changed either.

In [ ]:
@PrototypeV1Engine.depth_tracker
def _quiescence(
    self, board: chess.Board, alpha: Score, beta: Score
) -> PovScore:
    stand_pat = self.evaluator.get_score().relative

    if stand_pat >= beta:
        return PovScore(beta, board.turn)
    alpha = max(alpha, stand_pat)

    for move in board.generate_legal_captures():
        detailed_move = DetailedMove(board, move)

        if move.promotion is None:
            if self._can_delta_prune(stand_pat, alpha, detailed_move):
                continue
            if self._see_capture(board, detailed_move) < 0:
                continue

        board.push(move)
        self.evaluator.push(detailed_move)

        value = self._quiescence(board, -beta, -alpha).pov(not board.turn)

        self.evaluator.pop()
        board.pop()

        if value >= beta:
            return PovScore(value, board.turn)
        alpha = max(alpha, value)
    return PovScore(alpha, board.turn)


PrototypeV1Engine._quiescence = _quiescence


def _can_delta_prune(
    self, stand_pat: Score, alpha: Score, detailed_move: DetailedMove
) -> bool:
    POTENTIAL_POSITION_ADVANTAGE = 200
    piece_value = self.evaluator.piece_values[
        detailed_move.capturedPiece.piece.piece_type]
    best_alpha = Cp(
        stand_pat.score() + piece_value + POTENTIAL_POSITION_ADVANTAGE
    )
    return best_alpha < alpha


PrototypeV1Engine._can_delta_prune = _can_delta_prune


def _see_capture(self, board: chess.Board, detailed_move: DetailedMove) -> int:
    board.push(detailed_move.move)
    captured_piece = detailed_move.capturedPiece
    value = self.evaluator.piece_values[
        captured_piece.piece.piece_type
    ] - self._see(board, detailed_move.placedPiece.square)
    board.pop()
    return value


PrototypeV1Engine._see_capture = _see_capture


def _see(self, board: chess.Board, square: chess.Square) -> int:
    attackers_squares = [
        square for square in board.attackers(board.turn, square)
        if not board.is_pinned(board.turn, square)
    ]
    if not attackers_squares:
        return 0

    attacker_square = min(
        attackers_squares,
        key=lambda attackers_square: self.evaluator.piece_values[
            board.piece_type_at(attackers_square)]
    )
    captured_piece = board.piece_type_at(square)

    detailed_move = DetailedMove(
        board, chess.Move(from_square=attacker_square, to_square=square)
    )
    board.push(detailed_move.move)
    value = max(
        0,
        self.evaluator.piece_values[captured_piece] -
        self._see(board, detailed_move.placedPiece.square)
    )
    board.pop()
    return value


PrototypeV1Engine._see = _see

Next, the `analyse` method is implemented. The goal of this method is to evaluate all next moves and return them as a sorted list. To implement this two functions `_evaluate_nove` and `_evaluate_moves` are needed. All of these have been shown already in previous chapters, so there will be just short descriptions for a reminder.

The `_evaluate_move` function will score the given move by using the `_value` function. It will need the `depth_tracker` decorator as well.

In [ ]:
from chess.engine import Score, Cp, MateGiven
from converted_notebooks.s04_engine_interface import ScoredMove, LowestScore, HighestScore


@PrototypeV1Engine.depth_tracker
def _evaluate_move(
    self, board: chess.Board, move: chess.Move, depth: int
) -> ScoredMove:
    self.evaluator.push(DetailedMove(board, move))
    board.push(move)
    score = self._value(board, depth - 1, LowestScore, HighestScore).white()
    board.pop()
    self.evaluator.pop()

    return ScoredMove(score=score, move=move)


PrototypeV1Engine._evaluate_move = _evaluate_move

The `_evaluate_moves` implements the iterative deepening algorithm and thus will score all moves with the `_evaluate_move` method starting at a depth of 0. It will then successively increase the depth until the max look ahead depth is reached. 

In [ ]:
import logging


def _evaluate_moves(self, board: chess.Board) -> list[ScoredMove]:
    logging.info(f"Max depth: {self.max_look_ahead_depth}")
    self.cache.clear()
    self.evaluator.init(board)

    for depth in range(1, self.max_look_ahead_depth + 1):
        scored_moves = [
            self._evaluate_move(board, move, depth)
            for move in board.legal_moves
        ]

        logging.info(f"Depth {depth}")

    return scored_moves


PrototypeV1Engine._evaluate_moves = _evaluate_moves

Then `analyse` uses this method to get a list of scored moves and will return this sorted to the caller.

In [ ]:
def analyse(self, board: chess.Board) -> list[ScoredMove]:
    next_moves = self._evaluate_moves(board.copy())

    whites_turn = board.turn is chess.WHITE
    next_moves.sort(reverse=whites_turn)

    return next_moves


PrototypeV1Engine.analyse = analyse

Besides `analyse` the interface also requires us to implement the `play` function. In contrast to `analyse`, `play` should only return the best move. Therefore, the iterative deepening algorithm could call directly the `_value` function. The problem though is that `_value` gives us only the best score, but not the corresponding move. There are many possible solutions to this problem. One of them is to rewrite `_value` so it actually returns the move. This implementation instead makes use of the cache to find the best move afterwards. 

The `_find_move` method takes a chess board `board` and the score `score` of the best move as parameters. Its goal is to find a corresponding move by querying the cache and return it. It will therefore iterate over all moves and get the cache entries for these, if existent. Because the handling of Mate values is done *after* resetting the board, this handling needs to be done here too. To be the best move the cache entry type needs to be exact and the stored value must be the same as score negated. The negation is needed because the cache entry is for a position one move further than the passed board and therefore the perspective changed. There can be multiple best moves, therefore they are saved in a list and a random element is returned from it in the end.

In [ ]:
from converted_notebooks.s11_iterative_deepening import NodeType


def _find_move(self, board: chess.Board, score: PovScore) -> chess.Move:
    best_moves = []
    for move in board.legal_moves:
        board.push(move)
        cache_key = self.cache.get_key(board)
        board.pop()

        try:
            type, value = self.cache.load_cache(cache_key, self.max_look_ahead_depth - 1)

            if type == NodeType.EXACT and value.white() == score.white():
                best_moves.append(move)
        except KeyError:
            pass

    assert best_moves, f"No best move found with the given score {score.white()}"
    return random.choice(best_moves)


PrototypeV1Engine._find_move = _find_move

Now we can implement the iterative deepening algorithm for `play`. The method `_find_best_move` works similarly to `_evaluate_moves`. It will also take the chess board `board` as a parameter. But instead of returning a list of all scored moves, it will just return the best move. To do this it calls directly `_value` in its loop. At the end it uses the previously defined `_find_move` method to get the best move from the cache.

In [ ]:
from chess.engine import MateGiven

import logging


def _find_best_move(self, board: chess.Board) -> chess.Move:
    logging.info(f"Max depth: {self.max_look_ahead_depth}")
    self.cache.clear()
    self.evaluator.init(board)

    for depth in range(self.max_look_ahead_depth + 1):
        score = self._value(board, depth, LowestScore, HighestScore)
        logging.info(f"Depth {depth}")

    return self._find_move(board, score)


PrototypeV1Engine._find_best_move = _find_best_move

Lastly, the `play` method needs to be implemented. It will simply return a `chess.engine.PlayResult` with the best move found by the `_find_best_move` method.

In [ ]:
def play(self, board: chess.Board) -> chess.engine.PlayResult:
    return chess.engine.PlayResult(
        move=self._find_best_move(board.copy()), ponder=None
    )


PrototypeV1Engine.play = play

As usual, the engine is tested against the `middlegame_board`.

In [ ]:
import random

from converted_notebooks.s09_minimax_engine import middlegame_board
from converted_notebooks.s12_simplified_evaluation import incremental_simplified_evaluator

random.seed(42)

engine = PrototypeV1Engine(
    evaluator=incremental_simplified_evaluator, max_look_ahead_depth=4
)
engine.analyse(middlegame_board)

We test the added mate functionality by defining a method `run_mate_tests` that takes an engine `engine` as a parameter and asks it to solve a set of positions.

In [ ]:
import random
from chess import Move, Board
from chess.engine import Mate

from converted_notebooks.s04_engine_interface import ScoredMove

random.seed(42)

mate_tests = [(
    "3k4/6R1/8/4R3/8/8/8/7K w - - 0 1",
    [
        ScoredMove(score=Mate(+3), move=Move.from_uci('g7h7')),
        ScoredMove(score=Mate(+3), move=Move.from_uci('g7f7')),
        ScoredMove(score=Mate(+3), move=Move.from_uci('g7b7')),
        ScoredMove(score=Mate(+3), move=Move.from_uci('g7a7')),
        ScoredMove(score=Mate(+3), move=Move.from_uci('e5e7')),
        ScoredMove(score=Mate(+3), move=Move.from_uci('e5e6')),
        ScoredMove(score=Mate(+3), move=Move.from_uci('e5c5')),
        ScoredMove(score=Mate(+3), move=Move.from_uci('e5e4')),
        ScoredMove(score=Mate(+3), move=Move.from_uci('e5e3')),
        ScoredMove(score=Mate(+3), move=Move.from_uci('e5e2')),
        ScoredMove(score=Mate(+3), move=Move.from_uci('e5e1')),
        ScoredMove(score=Mate(+3), move=Move.from_uci('h1h2')),
        ScoredMove(score=Mate(+3), move=Move.from_uci('h1g2')),
        ScoredMove(score=Mate(+3), move=Move.from_uci('h1g1'))
    ]
),
              (
                  "2q3k1/2p1npp1/8/3N4/8/8/2Q4R/1B4K1 w - - 0 1",
                  [ScoredMove(score=Mate(+3), move=Move.from_uci('h2h8'))]
              ),
              (
                  "R1q3k1/2p1npp1/4P3/3N4/8/8/2Q5/1B4K1 w - - 0 1",
                  [
                      ScoredMove(score=Mate(+5), move=Move.from_uci('a8c8')),
                      ScoredMove(score=Mate(+5), move=Move.from_uci('c2h7'))
                  ]
              ),
              (
                  "2k5/8/2K2R2/8/8/8/8/8 b - - 0 1",
                  [ScoredMove(score=Mate(+6), move=Move.from_uci('c8b8'))]
              )]


def run_mate_tests(engine):
    for fen, best_moves in mate_tests:
        board = Board(fen)
        best_move = engine.analyse(board)[0]
        assert best_move in best_moves, f"{best_move} is not in {best_moves}"

In [ ]:
from converted_notebooks.s12_simplified_evaluation import incremental_simplified_evaluator

random.seed(42)

engine = PrototypeV1Engine(
    evaluator=incremental_simplified_evaluator, max_look_ahead_depth=6
)

run_mate_tests(engine)

It is also possible to play against the prototype engine with the human engine.

In [ ]:
import random
from converted_notebooks.s06_play import play_game
from converted_notebooks.s07_human_engine import HumanEngine
from converted_notebooks.s12_simplified_evaluation import incremental_simplified_evaluator

random.seed(42)

board = chess.Board()
play_game(
    board,
    HumanEngine(),
    PrototypeV1Engine(
        evaluator=incremental_simplified_evaluator, max_look_ahead_depth=3
    ),
    display_board=True,
    log_moves=True
)
print(board.outcome())